In [33]:
import xml.sax
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [42]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amankumarkashyap/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
count_of_pages = 0
Identities = {}
stop_words=set(stopwords.words('english'))

In [44]:
def get_infobar(data):
    data=data.split('\n')
    length=len(data)
    ret=[]
    flag=0
    for i in range(length):
        if flag%2==0 and re.search(r'\{\{infobox',data[i]):
            flag=1
            ret.append(re.sub(r'\{\{infobox','',data[i]))
        elif flag%2==1:
            if data[i]=='}}':
                flag=flag+1
                continue
            ret.append(data[i])
        elif flag==0:
            if i*100/length > 50:
                break

    return final_processing(' '.join(ret))

def get_categories(data):
    data=re.findall(r'\[\[category:.*\]\]',data)
    ret=[]
    for i in data:
        ret.append(i[11:len(i)-2])

    return final_processing(' '.join(ret))

def get_external_links(text):
    data=text.split('==external links==')
    if len(data)==1:
        data=text.split('==external links ==')
    if len(data)==1:
        data=text.split('== external links==')
    if len(data)==1:
        data=text.split('== external links ==')
    if len(data)==1:
        return []
    data=data[1]
    data=re.findall(r'\*\s*\[.*\]',data)
    ret=[]
    for i in data:
        ret.append(i[2:len(i)-1])
    return final_processing(' '.join(ret))

def get_references(text):
    ret=[]
    data1=re.findall(r'\|\s*title[^\|]*',text)
    for i in data1:
        ret.append(i[i.find('=')+1:len(i)-1])
    return final_processing(' '.join(ret))

In [45]:
def final_processing(data):
    data=data.strip().encode("ascii",errors="ignore").decode()
    #data=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',' ',data)
    data=re.sub(r'\`|\~|\!|\@|\#|\"|\'|\$|\%|\^|\&|\*|\(|\)|\-|\_|\=|\+|\\|\||\]|\[|\}|\{|\;|\:|\/|\?|\.|\>|\,|\<|\'|\n|\||\|\/"',r' ',data)
    data=re.sub(r'&nbsp;|&lt;|&gt;|&amp;|&quot;|&apos;|&cent;|&pound;|&yen;|&euro;|&copy;|&reg;',r' ',data)
    #Above code removes any special character
    final_data=[]
    ss=SnowballStemmer('english')
    data=data.split()
    for w in data:
        if not w.strip() in stop_words:
            w=ss.stem(w)
            final_data.append(w)
    return final_data

In [46]:
def process_text(text,title):
    text=text.lower()
    references=[]
    categories=[]
    external_links=[]
    data=text.split('==references==')
    if data[0]==text:
        data=text.split('==references ==')
    if data[0]==text:
        data=text.split('== references==')
    if data[0]==text:
        data=text.split('== references ==')
    if data[0]==text:
        categories=get_categories(data[0])
        external_links=get_external_links(data[0])
    else:
        categories=get_categories(data[1])
        external_links=get_external_links(data[1])
        references=get_references(data[1])	

    info_bar=get_infobar(data[0])
    data[0]=re.sub(r'\{\{.*\}\}',r' ',data[0])
    body=final_processing(data[0])
    title=final_processing(title.lower())
    return title,body,info_bar,categories,external_links,references

In [47]:
class MyParser(xml.sax.ContentHandler):
    def __init__(self):
        self.title = ""
        self.tag = ""
        self.id = ""
        self.text = ""
        self.idFlag = 0 
    def characters(self, content):
        if self.tag == 'text':
            self.text+=content
        if self.tag == 'title':
            self.title+=content
        if self.tag == 'id' and self.idFlag==0:
            self.id = content
            self.idFlag=1
    def startElement(self,name,attrs):
        self.tag=name

    def endElement(self,name):
        global count_of_pages
        if name=='page':
            self.title=self.title.strip().encode("ascii",errors="ignore").decode()
            Identities[count_of_pages]=self.title
            title,body,infobox,categories,external_links,references=process_text(self.text,self.title)
            print(title)
            print(body)
            print(categories)
            print(external_links)
            print(references)
            self.tag=""
            self.title=""
            self.text=""
            self.id=""
            self.idFlag=0

In [48]:
def main():
    global count_of_pages
    global Identities
    parser = xml.sax.make_parser()
    parser.setFeature(xml.sax.handler.feature_namespaces, False)
    handler = MyParser()
    parser.setContentHandler(handler)
    output = parser.parse('./dummy.xml')
    print(count_of_pages)

In [49]:
main()

['accessiblecomput']
['redirect', 'comput', 'access']
[]
[]
[]
['anarch']
['anarch', 'polit', 'philosophi', 'polit', 'movement', 'movement', 'reject', 'involuntari', 'coerciv', 'form', 'hierarchi', 'call', 'abolit', 'state', 'politi', 'state', 'hold', 'undesir', 'unnecessari', 'harm', 'histori', 'anarch', 'goe', 'back', 'prehistori', 'human', 'live', 'anarchist', 'societi', 'long', 'establish', 'formal', 'state', 'realm', 'empir', 'rise', 'organis', 'hierarch', 'bodi', 'skeptic', 'toward', 'author', 'also', 'rose', '19th', 'centuri', 'self', 'conscious', 'polit', 'movement', 'emerg', 'latter', 'half', '19th', 'first', 'decad', '20th', 'centuri', 'anarchist', 'movement', 'flourish', 'part', 'world', 'signific', 'role', 'worker', 'struggl', 'emancip', 'various', 'anarchist', 'school', 'thought', 'form', 'period', 'anarchist', 'taken', 'part', 'sever', 'revolut', 'notabl', 'spanish', 'civil', 'war', 'whose', 'end', 'mark', 'end', 'classic', 'era', 'anarch', 'anarch', 'classic', 'era', 'la